### Задачи к Лекции 4

__Исходные данные__

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("mlbootcamp5_train (1).csv",
                 sep=";",
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [ ]:
# A lot of code here


In [ ]:
target = df['cardio']

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
from sklearn.model_selection import train_test_split

train, test, target_train, target_test = train_test_split(
    df[['age','height','weight','ap_lo','ap_hi']], target,
    test_size=0.3)

In [ ]:
# Обучаем модель на тренировочной выборке
tree = DecisionTreeClassifier(max_depth=6, random_state=13)
tree.fit(train, target_train);
predict = tree.predict(test)

from sklearn.metrics import accuracy_score
accuracy_score(predict, target_test)

0.7221904761904762

In [ ]:
from sklearn.model_selection import cross_validate, cross_val_score
data= df[['age','height','weight', 'ap_hi', 'ap_lo']]
target = df['cardio']
tree = DecisionTreeClassifier(max_depth=10, random_state=13)
cv = cross_val_score(tree,
                    df[['age','height','weight', 'ap_hi', 'ap_lo']],
                    target,
                    n_jobs=2,
                    scoring='accuracy',
                    cv=3)
cv, cv.mean()

(array([0.7139796 , 0.71606737, 0.71666738]), np.float64(0.7155714513121533))

In [ ]:
tree = DecisionTreeClassifier(max_depth=50, random_state=13)
cv = cross_val_score(tree,
                    df[['age','height','weight', 'ap_hi', 'ap_lo']],
                    target,
                    n_jobs=2,
                    scoring='accuracy',
                    cv=3)
cv, cv.mean()

(array([0.62102511, 0.6208803 , 0.62456607]), np.float64(0.6221571590292209))

In [ ]:
train, test, target_train, target_test = train_test_split(
    df[['age','height','weight', 'ap_hi', 'ap_lo']], target,
    test_size=0.3
)

In [ ]:
%%time
from sklearn.model_selection import GridSearchCV
params = {
    'max_depth': np.arange(2, 10, 1),
    'min_samples_split': [2, 4, 8, 16],
}
tree = DecisionTreeClassifier(random_state=13)
grid = GridSearchCV(
    tree, params, scoring='accuracy',
    cv=3,
)
grid.fit(train, target_train);

CPU times: user 6.04 s, sys: 0 ns, total: 6.04 s
Wall time: 7.81 s


GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=13),
             param_grid={'max_depth': array([2, 3, 4, 5, 6, 7, 8, 9]),
                         'min_samples_split': [2, 4, 8, 16]},
             scoring='accuracy')

In [ ]:
grid.best_score_

np.float64(0.7196122913106833)

In [ ]:
grid.best_params_

{'max_depth': np.int64(4), 'min_samples_split': 2}

In [ ]:
predict = grid.best_estimator_.predict(test)
accuracy_score(target_test, predict)

0.7265238095238096

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
data = df[['age', 'height', 'weight','ap_lo', 'ap_hi']].values
target = df['cardio']
from sklearn.model_selection import train_test_split
# Разделение данных на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2)

def plot_scatter(cls, data, target, outer=0.1, **kargs):
    x_min, x_max = data[:, 0].min(), data[:, 0].max()
    x_w = x_max - x_min
    y_min, y_max = data[:, 1].min(), data[:, 1].max()
    y_w = y_max - y_min
    xx, yy = np.meshgrid(
        np.arange(x_min - x_w * outer, x_max + x_w * outer, 0.05),
        np.arange(y_min - y_w * outer, y_max + y_w * outer, 0.05),
    )

    model = cls(**kargs)
    model.fit(data, target)
    predict = model.predict(np.stack([xx.ravel(), yy.ravel()], axis=1))
    plt.pcolormesh(xx, yy, predict.reshape(xx.shape), cmap=plt.get_cmap("Wistia"))
    plt.scatter(data[:, 0], data[:, 1], c=target,
                cmap=plt.get_cmap("Wistia"),
                edgecolors='k', s=50)
    plt.title('K-Nearest Neighbors Classifier')
    plt.xlabel('Feature 1')
    plt.ylabel('Feature 2')
    plt.show()




In [ ]:
from sklearn.neighbors import KNeighborsClassifier
plot_scatter(KNeighborsClassifier, data, target, 0.2,
             n_neighbors=25)

In [ ]:
knn_classifier = KNeighborsClassifier(n_neighbors=25)
plot_scatter(KNeighborsClassifier,
             X_train,
             y_train,
             n_neighbors=25)

**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [ ]:
# Your code here


In [ ]:
from sklearn.metrics import confusion_matrix
predict_proba = knn_classifier.predict_proba(train)
predict = knn_classifier.predict(train)
cnf_matrix = confusion_matrix(target, predict)

In [ ]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Матрица неточностей', cmap=plt.cm.Blues):
    import itertools

    if normalize:
        cm = cm.astype('float')/cm.sum(axis=0)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.min() + (cm.max() - cm.min()) * 2 / 3.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('Истина')
    plt.xlabel('Предсказание')
    plt.tight_layout()

In [ ]:
plot_confusion_matrix(cnf_matrix, ["Класс 0", "Класс 1"], normalize=True)

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(target, predict, target_names=["Здоров", "Болен"])
print(report)

In [ ]:
from sklearn.metrics import roc_curve
fpr, tpr, thr = roc_curve(target, predict_proba[:,1], pos_label=1)

**Комментарии:** Ваши комментарии здесь.